## Project 2 
This jupyter notebook file provides strong hints and guidance for setting up simulations of cart-pole system in Drake.

First, we need to import packages

In [ ]:
server_args = []
import math
import numpy as np
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)
from pydrake.all import (LeafSystem,BasicVector,DiagramBuilder, AddMultibodyPlantSceneGraph, Parser, LinearQuadraticRegulator,
                         Simulator, RigidTransform, CoulombFriction, FindResourceOrThrow, DrakeVisualizer, ConnectContactResultsToDrakeVisualizer,
                         RollPitchYaw, JointIndex, namedview, ConnectMeshcatVisualizer,
                         Value, List, ZeroOrderHold, SpatialAcceleration, RotationMatrix, AbstractValue, ConstantVectorSource)


# if you need to use other class from Drake, you can add them to the list above

## Construct block diagram, add the cart-pole model from urdf file 
- For testing purpose, we add null controller that always send it zero to the plant

In [ ]:
builder = DiagramBuilder()
# First add the cart-pole system from a urdf file
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=T)
urdf_path = "./urdfExample_cart_pole.urdf"    
cart_pole = Parser(plant, scene_graph).AddModelFromFile(urdf_path)    
plant.Finalize()

# Add controller (u = 0)
controller = builder.AddSystem(ConstantVectorSource([0]))

# connect to make diagram
builder.Connect(controller.get_output_port(), plant.get_actuation_input_port())

# set up visualization using meshcat
meshcat = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, open_browser=True)
diagram = builder.Build()

# start simulation
UprightState = np.array([0, np.pi, 0, 0])   # the state of the cart-pole is organized as [z, zdot, theta, theta_dot]
simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1)
context = simulator.get_mutable_context()
context.SetContinuousState(UprightState + np.array([0.1,0.3,0.3,0.1]))  
simulator.Initialize()

sim_time = 5
meshcat.start_recording() 
simulator.AdvanceTo(sim_time)
meshcat.stop_recording()
meshcat.publish_recording()
simulator.AdvanceTo(sim_time)

# Design and testing
- You should design your feedback gain K and test its performance in the simulation 
- Note that the plant is represented in continuous time model, but our design was based on discrete time linearized model (with sampling time T), you need to add the zero-order-holder between the feedback gain block and the plant. In other words, the control action will be sent to the plant every T second (not continuously)
- In the simulator, you can get everything from the plant model. In particular, we get all the state vector from the plant using "get_state_output_port()". This is a special port for the MultibodyPlant class (this class of systems can be defined from the urdf file)
- If you want to use only part of the full state, say [$\theta$, $\dot\theta$], then we can extract these info from the full state info. 

In [ ]:
# Define your own controller in discrete time
# Define the system.
class state_feedback(LeafSystem):
    def __init__(self, T,gain):
        LeafSystem.__init__(self)        
        num_input = 4
        num_output = 1        
        # Define the input
        self.DeclareVectorInputPort("x_meas", BasicVector(num_input))
        # Define the output
        self.DeclareVectorOutputPort("control", BasicVector(num_output), self.CalcOutputY)
        self.gain = gain        
    def CalcOutputY(self, context, output):        
        x_meas = self.get_input_port(0).Eval(context)

        x_theta = np.array([[x_meas[1]],[x_meas[3]]])        
        y = self.gain@x_theta
        output.SetFromVector(y)

In [ ]:
from pydrake.all import ZeroOrderHold

builder = DiagramBuilder()
# First add the cart-pole system from a urdf file
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
urdf_path = "./urdfExample_cart_pole.urdf"    
cart_pole = Parser(plant, scene_graph).AddModelFromFile(urdf_path)    
plant.Finalize()

# Add controller (you need to design your feedback gain K first)
T = 0.005
K = np.array([[1,2]])  #this is just an example
controller = builder.AddSystem(state_feedback(T,-K))
zoh = builder.AddSystem(ZeroOrderHold(T,1))  # add the zoh to simulate the discrete update effect

# connect to make diagram
builder.Connect(plant.get_state_output_port(), controller.get_input_port(0))  # get_state_output_port()  will return the full state
# in this case, the full state =[z,theta, zdot, theta_dot] 
builder.Connect(zoh.get_output_port(0), plant.get_actuation_input_port())
builder.Connect(controller.get_output_port(0), zoh.get_input_port(0))

# set up visualization using meshcat
meshcat = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, open_browser=True)
diagram = builder.Build()

# start simulation
UprightState = np.array([0, np.pi, 0, 0])   # the state of the cart-pole is organized as [z, zdot, theta, theta_dot]
simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1)
context = simulator.get_mutable_context()
context.SetContinuousState(UprightState + np.array([0.1,0.3,0.3,0.1]))  
simulator.Initialize()

sim_time = 5
meshcat.start_recording() 
simulator.AdvanceTo(sim_time)
meshcat.stop_recording()
meshcat.publish_recording()
simulator.AdvanceTo(sim_time)